# 基于 T5 的文本摘要

## step1 import related lib

- `Seq2SeqTrainer`, `Seq2SeqTrainingArguments` 专门针对 `seq2seq` 任务优化

- `seq2seq` 任务的一些特殊要求:
    - **自定义损失函数：** 通常是交叉熵函数，但某些情况需要自定义
    - **生成任务：** 在评估和预测阶段，Seq2Seq模型需要生成输出序列，这涉及到解码过程，而不仅仅是分类或回归
    - **数据处理：** Seq2Seq任务的数据处理可能需要特殊的预处理和后处理步骤，例如对输入和输出序列进行填充和截断

In [1]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

## step2 load datases

这里的数据 `nlpcc_2017` 是裁剪过的，原本的实在太大了，为了做实验方便，较少样本数量

In [2]:
ds = Dataset.load_from_disk("./nlpcc_2017/")
ds

Dataset({
    features: ['title', 'content'],
    num_rows: 5000
})

In [3]:
ds = ds.train_test_split(100, seed=42)  # 这里加 `seed` 是为了方便后续区分 glm 模型，至于为什么是42，可能是因为《银河系漫游指南》说过宇宙的终极答案是 42
ds

DatasetDict({
    train: Dataset({
        features: ['title', 'content'],
        num_rows: 4900
    })
    test: Dataset({
        features: ['title', 'content'],
        num_rows: 100
    })
})

In [4]:
ds['train'][0]

{'title': '组图:黑河边防军人零下30℃户外训练,冰霜沾满眉毛和睫毛,防寒服上满是冰霜。',
 'content': '中国军网2014-12-1709:08:0412月16日,黑龙江省军区驻黑河某边防团机动步兵连官兵,冒着-30℃严寒气温进行体能训练,挑战极寒,锻造钢筋铁骨。该连素有“世界冠军的摇篮”之称,曾有5人24人次登上世界军事五项冠军的领奖台。(魏建顺摄)黑龙江省军区驻黑河某边防团机动步兵连官兵冒着-30℃严寒气温进行体能训练驻黑河某边防团机动步兵连官兵严寒中户外训练,防寒服上满是冰霜驻黑河某边防团机动步兵连官兵严寒中户外训练,防寒服上满是冰霜官兵睫毛上都被冻上了冰霜官兵们睫毛上都被冻上了冰霜驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练'}

## step3 process data

`AutoTokenizer` 加载不了 `Langboat/mengzi-t5-base` 的 `tokenizer`

In [5]:
from transformers import T5Tokenizer
model_path = r"D:\CodeLibrary\huggingface_model\Langboat\mengzi-t5-base"
# tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)
tokenizer

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


T5Tokenizer(name_or_path='D:\CodeLibrary\huggingface_model\Langboat\mengzi-t5-base', vocab_size=32028, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<ex

`seq2seq` 任务中，需要自定义处理函数：
- 添加提示词前缀（类似于 prompt）
- 将输入（inputs）和输出（labels）序列进行编码
- 对输入和输出序列进行填充和截断

In [8]:
def process_func(examples):
    contents = ["生成摘要: \n" + e for e in examples['content']]
    inputs = tokenizer(examples['content'], max_length=384, truncation=True)
    labels = tokenizer(examples['title'], max_length=64, truncation=True)
    inputs['labels'] = labels['input_ids']

    return inputs

In [9]:
tokenized_ds = ds.map(process_func, batched=True)
tokenized_ds

Map:   0%|          | 0/4900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'content', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4900
    })
    test: Dataset({
        features: ['title', 'content', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

In [10]:
# 经过 tokenzier 后的数据都带有 eos token (</s>)
tokenizer.decode(tokenized_ds['train'][0]['input_ids'])

'中国军网2014-12-1709:08:0412月16日,黑龙江省军区驻黑河某边防团机动步兵连官兵,冒着-30°C严寒气温进行体能训练,挑战极寒,锻造钢筋铁骨。该连素有“世界冠军的摇篮”之称,曾有5人24人次登上世界军事五项冠军的领奖台。(魏建顺摄)黑龙江省军区驻黑河某边防团机动步兵连官兵冒着-30°C严寒气温进行体能训练驻黑河某边防团机动步兵连官兵严寒中户外训练,防寒服上满是冰霜驻黑河某边防团机动步兵连官兵严寒中户外训练,防寒服上满是冰霜官兵睫毛上都被冻上了冰霜官兵们睫毛上都被冻上了冰霜驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练驻黑河某边防团机动步兵连官兵严寒中进行户外体能训练</s>'

In [11]:
tokenizer.decode(tokenized_ds['train'][0]['labels'])

'组图:黑河边防军人零下30°C户外训练,冰霜沾满眉毛和睫毛,防寒服上满是冰霜。</s>'

## step4 create model

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

## step5 create eval func

模型的输出是 logits, 评估指标用的是 Rouge，所以需要decode转为 **汉字**  在进行评估指标计算

In [14]:
import numpy as np
from rouge_chinese import Rouge

rouge = Rouge() # 创建评估指标

def compute_metric(evalPred):
    predictions, labels = evalPred
    decode_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # labels中很多 是 -100 的pad token，转换成 pad token id（正数）
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # 将decode后的数据按英文那样 空格 隔开，好后续分词做rouge（貌似不需要这一步也可以计算rouge）
    decode_preds = [" ".join(p) for p in decode_preds]
    decode_labels = [" ".join(l) for l in decode_labels]

    # 计算scores(平均分数)
    scores = rouge.get_scores(decode_preds, decode_labels, avg=True)

    return {
        "rouge-1": scores['rouge-1']['f'],  # 取 f1 scores
        "rouge-2": scores['rouge-2']['f'],
        "rouge-l": scores['rouge-l']['f']
    }


## step6 create args

`predict_with_generate` 告诉模型在评估和预测时使用生成（generation）的方式来生成输出序列，而不是简单地使用模型的输出层


启用功能：
- **生成输出序列**：在评估和预测阶段，模型会使用诸如束搜索（beam search）或贪心搜索（greedy search）等生成策略来生成输出序列。这对于像机器翻译、文本摘要等任务非常重要，因为这些任务需要模型生成连贯且有意义的文本。

- **计算生成指标：** 启用这个参数后，模型可以计算一些与生成相关的指标，如BLEU、ROUGE等，这些指标通常用于评估生成文本的质量。

- **兼容性：** 许多预训练的seq2seq模型（如T5、BART等）在设计时就考虑了生成任务，因此使用predict_with_generate=True 可以确保模型在评估和预测时能够正确地使用这些生成策略。

In [13]:
args = Seq2SeqTrainingArguments(
    output_dir="./summary",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    logging_steps=8,
    eval_strategy='epoch',
    save_strategy='epoch',
    num_train_epochs=1,
    predict_with_generate=True # seq2seq 模型一定要加
)

## step7 create trainer

In [15]:
trainer = Seq2SeqTrainer(
    args=args,
    model=model,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metric,
    data_collator=DataCollatorForSeq2Seq(tokenizer)
)

## step8 model train

In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
0,2.634000,2.330518,0.486443,0.314908,0.408104


e:\software\anaconda3\envs\transformers\Lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=153, training_loss=3.066711416431502, metrics={'train_runtime': 772.5064, 'train_samples_per_second': 6.343, 'train_steps_per_second': 0.198, 'total_flos': 1963761044484096.0, 'train_loss': 3.066711416431502, 'epoch': 0.9991836734693877})

## step9 model inference

In [17]:
from transformers import pipeline

pipe = pipeline('text2text-generation', model=model, tokenizer=tokenizer, device=0)

In [18]:
pipe("摘要生成:\n" + ds["test"][-1]["content"], max_length=64, do_sample=True)

[{'generated_text': '中船重工拟对其相关业务进行调整或涉及公司发行股份;中船重工拟对外投资,涉及定增等交易,涉及公司股票停牌'}]

In [19]:
ds["test"][-1]["title"]

'中国重工拟以持有的动力相关资产进行对外投资,参与中船重工拟打造的动力业务平台公司,将继续停牌。'